# Get Data Multi database

In [ ]:
import psycopg2
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

## Local Postgres

In [ ]:
conn_local_postgres = psycopg2.connect(
  host="localhost",
  port="5432",
  database="prosain171",
  user="postgres",
  password="root"
)

In [ ]:
df_local_postgres = pd.read_sql_query('select * from sepal_length', con=conn_local_postgres)
conn_local_postgres.close()

In [ ]:
df_local_postgres

,id,sepal_length
0,1,5.1
1,2,4.9
2,3,4.7
3,4,4.6
4,5,5.0
...,...,...
145,146,6.7
146,147,6.3
147,148,6.5
148,149,6.2


## Cloud Postgres

In [ ]:
conn_cloud_postgres = psycopg2.connect(
  host="floppy.db.elephantsql.com",
  port="5432",
  database="tmcbpwlb",
  user="tmcbpwlb",
  password="sPtnOpoHfBD-7DpfJltLzWwbcHJjRRrY"
)

In [ ]:
df_cloud_postgres = pd.read_sql_query('select * from "sepal_width"', con=conn_cloud_postgres)
conn_cloud_postgres.close()

In [ ]:
df_cloud_postgres

,id,sepal_width
0,1,3.5
1,2,3.0
2,3,3.2
3,4,3.1
4,5,3.6
...,...,...
145,146,3.0
146,147,2.5
147,148,3.0
148,149,3.4


## Local Mysql

In [ ]:
_= !pip install mysql-connector

In [ ]:
import mysql.connector

In [ ]:
conn_local_mysql = mysql.connector.connect(
  host="localhost",
  port="3306",
  database="prosain171",
  user="root",
  password=""
)

In [ ]:
df_local_mysql = pd.read_sql_query('select * from petal_length', con=conn_local_mysql)
conn_local_mysql.close()

In [ ]:
df_local_mysql

,id,petal_length
0,1,1.4
1,2,1.4
2,3,1.3
3,4,1.5
4,5,1.4
...,...,...
145,146,5.2
146,147,5.0
147,148,5.2
148,149,5.4


## Local Sql-Server

In [ ]:
import pyodbc

In [ ]:
conn_local_sqlserver = pyodbc.connect(
  "Driver={ODBC Driver 17 for SQL Server};"
  "Server=farisulhaq\SQLEXPRESS;"
  "Trusted_Connection=yes;"
)

In [ ]:
df_local_sqlserver = pd.read_sql_query("SELECT * FROM [prosain171].[dbo].[petal_width]", con=conn_local_sqlserver)
# conn_local_postgres.close()

In [ ]:
df_local_sqlserver

,id,petal_width,class
0,1,0.2,Iris-setosa
1,2,0.2,Iris-setosa
2,3,0.2,Iris-setosa
3,4,0.2,Iris-setosa
4,5,0.2,Iris-setosa
...,...,...,...
145,146,2.3,Iris-virginica
146,147,1.9,Iris-virginica
147,148,2.0,Iris-virginica
148,149,2.3,Iris-virginica


## Join antar database

In [ ]:
df = df_local_postgres.merge(df_cloud_postgres, on='id').merge(df_local_mysql, on='id').merge(df_local_sqlserver, on='id').drop('id', axis=1)

In [ ]:
df

,sepal_length,sepal_width,petal_length,petal_width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


## SSH


In [ ]:
_= !pip install sshtunnel

In [ ]:
import logging
import sshtunnel
from sshtunnel import SSHTunnelForwarder

In [ ]:
ssh_host = '68.183.191.223'
ssh_username = 'prosaindata'
ssh_password = '+jS+1Gw(U7dnVl32M1'
database_username = 'prosaindata'
database_password = '+jS+1Gw(U7dnVl32M1'
database_name = 'prosaindata_db'
localhost = '127.0.0.1'

In [ ]:
def open_ssh_tunnel(verbose=False):  
  if verbose:
    sshtunnel.DEFAULT_LOGLEVEL = logging.DEBUG
  tunnel = SSHTunnelForwarder(
    (ssh_host, 22),
    ssh_username = ssh_username,
    ssh_password = ssh_password,
    remote_bind_address = ('127.0.0.1', 3306)
  )
  tunnel.start()
  return tunnel

def close_ssh_tunnel(tunnel):
  tunnel.close

def mysql_connect(tunnel):
  connection = mysql.connector.connect(
    host='127.0.0.1',
    user=database_username,
    password=database_password,
    database=database_name,
    port=tunnel.local_bind_port
  )
  return connection

def mysql_disconnect(connection):
  connection.close()

def run_query(sql, connection):
  return pd.read_sql_query(sql, connection)

In [ ]:
tunnel = open_ssh_tunnel()
conn = mysql_connect(tunnel)
df = run_query("SELECT * FROM sepal", conn)
mysql_disconnect(tunnel)
close_ssh_tunnel(tunnel)

In [ ]:
df

,id,sepallength,sepalwidth
0,1,5.1,3.5
1,2,4.9,3.0
2,3,4.7,3.2
3,4,4.6,3.1
4,5,5.0,3.6
...,...,...,...
145,146,6.7,3.0
146,147,6.3,2.5
147,148,6.5,3.0
148,149,6.2,3.4
